In [5]:
# Explore production and stock data
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pickle

In [6]:
# Production data Load  
data_prod= pd.read_csv('../Data/Production_Crops_Livestock_E_All_Data_(Normalized)/Production_Crops_Livestock_E_All_Data_(Normalized).csv', sep=',',low_memory=False, encoding='latin1',index_col=0).reset_index()
data_prod.columns = data_prod.columns.str.lower().str.replace(' ', '_')

In [7]:
flag_proc = 'processed'

if flag_proc == 'processed':
    data_trade = pd.read_pickle('../Data/Data_food_groups_code.pkl').reset_index()
    all_countries = set(data_trade.origin_country).union(data_trade.destin_country)
    print('Using processed dataset (Data_food_groups_code.pkl)')
else: 
    data_trade = pd.read_csv('../Data/raw_trade/Trade_DetailedTradeMatrix_E_All_Data_(Normalized).csv', sep=',', encoding='latin-1',index_col=0).reset_index()
    data_trade.columns = data_trade.columns.str.lower().str.replace(' ', '_')
    all_countries = set(data_trade.reporter_country).union(data_trade.partner_country)
    print('Using raw dataset (Trade_DetailedTradeMatrix_E_All_Data_(Normalized).csv)')


Using processed dataset (Data_food_groups_code.pkl)


In [8]:
data_trade

,index,destin_country_ISO,origin_country_ISO,item,item_code,year,unit,value,origin_country,destin_country,L1_foodex,Food_group
0,0,BY,UY,"Edible offals of horses and other equines, fr...",1098,2013,tonnes,7.0,Uruguay,Belarus,A01QR,Meat and meat products
1,1,BY,UY,"Edible offals of horses and other equines, fr...",1098,2013,1000 US$,8.0,Uruguay,Belarus,A01QR,Meat and meat products
2,2,BY,UY,"Horse meat, fresh or chilled",1097,2019,tonnes,49.6,Uruguay,Belarus,A01QR,Meat and meat products
3,3,BY,UY,"Horse meat, fresh or chilled",1097,2019,1000 US$,124.0,Uruguay,Belarus,A01QR,Meat and meat products
4,4,BY,UY,Husked rice,28,2007,tonnes,15.0,Uruguay,Belarus,A011X,"Legumes, nuts, oilseeds and spices"
...,...,...,...,...,...,...,...,...,...,...,...,...
28200314,32426007,ER,BR,Refined sugar,164,2015,1000 US$,913.0,Brazil,Eritrea,A032F,"Sugar and similar, confectionery and water-bas..."
28200315,32426008,ER,BR,Refined sugar,164,2016,1000 US$,22366.0,Brazil,Eritrea,A032F,"Sugar and similar, confectionery and water-bas..."
28200316,32426009,ER,BR,Refined sugar,164,2017,1000 US$,15325.0,Brazil,Eritrea,A032F,"Sugar and similar, confectionery and water-bas..."
28200317,32426010,ER,BR,Refined sugar,164,2020,1000 US$,4314.0,Brazil,Eritrea,A032F,"Sugar and similar, confectionery and water-bas..."


In [9]:
# FILTER PRODUCTION: remove areas that are not countries
data_prod_countries = data_prod.loc[(data_prod.area.isin(all_countries)),:]
len(data_prod.area.unique())

drop_geom = set(data_prod.area.unique()).difference(all_countries)


def Standardise_names(data):
    # Fix formatting isseues 
    data.loc[:,'area']= data.loc[:,'area'].replace('Micronesia','Micronesia (Federated States of)')
    data.loc[:,'area']=data.loc[:,'area'].replace('China','China, mainland')
    data.loc[:,'area']=data.loc[:,'area'].replace('Polynesia','French Polynesia')
    data.loc[:,'area']=data.loc[:,'area'].replace('Palestine','State of Palestine')
    data.loc[:,'area']=data.loc[:,'area'].replace("Ethiopia PDR",'Ethiopia')
    
    return data

# Standardise country names
data_prod_std = Standardise_names(data_prod_countries)

# Keep only years in TRADE data 
data_prod_std = data_prod_std.loc[(data_prod_std.year>=1986) & (data_prod_std.year<=2021),:]

In [10]:
print('Num of "item_codes" in data_prod:',len(data_prod_std.item_code.unique()))
print('Num of "item" in data_prod:',len(data_prod_std.item.unique()))

print('Num of "item_code" in data_trade:',len(data_trade.item_code.unique()))
print('Num of "item" in data_trade,', len(data_trade.item.unique()))

Num of "item_codes" in data_prod: 301
Num of "item" in data_prod: 301
Num of "item_code" in data_trade: 431
Num of "item" in data_trade, 431


In [11]:
print('Food products in data_prod AND in data_trade:',len(set(data_prod_std.item_code.unique()).intersection(data_trade.item_code.unique())))
print('Food products in data_prod AND in data_trade:',len(set(data_prod_std.item.unique()).intersection(data_trade.item.unique())))

Food products in data_prod AND in data_trade: 234
Food products in data_prod AND in data_trade: 234


In [12]:
not_intersected = set(data_trade.item).difference(data_prod_std.item.unique())
print('Food products in data_trade not in data_prod:',len(not_intersected))

not_intersected

Food products in data_trade not in data_prod: 197


{'Almonds, shelled',
 'Animal oils and fats n.e.c.',
 'Animal or vegetable fats and oils and their fractions, chemically modified, except those hydrogenated, inter-esterified, re-esterified or elaidinized, inedible mixtures or preparations of animal or vegetable fats or oils',
 'Apple juice',
 'Apple juice, concentrated',
 'Apricots, dried',
 'Barley flour and grits',
 'Barley, pearled',
 'Beet pulp',
 'Bovine meat, salted, dried or smoked',
 'Bran of barley',
 'Bran of buckwheat',
 'Bran of cereals n.e.c.',
 'Bran of maize',
 'Bran of millet',
 'Bran of mixed grain',
 'Bran of oats',
 'Bran of pulses',
 'Bran of rice',
 'Bran of rye',
 'Bran of sorghum',
 'Bran of wheat',
 'Brazil nuts, shelled',
 'Bread',
 'Breakfast cereals',
 'Bulgur',
 'Butter of karite nuts',
 'Buttermilk, curdled and acidified milk',
 'Cane sugar, non-centrifugal',
 'Canned mushrooms',
 'Cashew nuts, shelled',
 'Cassava, dry',
 'Cattle, butcher fat',
 'Cereal preparations',
 'Chocolate products nes',
 'Cider and

In [13]:
not_intersected = set(data_prod_std.item).difference(data_trade.item.unique())
#not_intersected = set(data_prod.item_code).difference(data_trade.item_code.unique())

print('Food products in data_prod not in data_trade:',len(not_intersected))
not_intersected

Food products in data_prod not in data_trade: 67


{'Abaca, manila hemp, raw',
 'Agave fibres, raw, n.e.c.',
 'Beef and Buffalo Meat, primary',
 'Bees',
 'Beeswax',
 'Buffalo fat, unrendered',
 'Butter and Ghee',
 'Butter of buffalo milk',
 'Cassava leaves',
 'Cattle and Buffaloes',
 'Cereals, primary',
 'Cheese (All Kinds)',
 'Cheese from milk of buffalo, fresh or processed',
 'Citrus Fruit, Total',
 'Coir, raw',
 'Cotton lint, ginned',
 'Edible offals of camels and other camelids, fresh, chilled or frozen',
 'Eggs Primary',
 'Evaporated & Condensed Milk',
 'Fibre Crops, Fibre Equivalent',
 'Flax, raw or retted',
 'Fruit Primary',
 'Goat fat, unrendered',
 'Green tea (not fermented), black tea (fermented) and partly fermented tea, in immediate packings of a content not exceeding 3 kg',
 'Jojoba seeds',
 'Jute, raw or retted',
 'Kapok fibre, raw',
 'Kapok fruit',
 'Kenaf, and other textile bast fibres, raw or retted',
 'Meat of mules, fresh or chilled',
 'Meat of other domestic camelids, fresh or chilled',
 'Meat of other domestic rode

In [14]:
# KEEP BASIC PRODUCTS: 
to_keep = set(data_prod_std.item.unique()).intersection(data_trade.item.unique()) # keep products in 

data_prod_keep = data_prod_std.loc[data_prod_std.item.isin(to_keep),:]
data_trade_keep = data_trade.loc[(data_trade.item.isin(to_keep)),:] #& (data_trade.unit =='1000 US$')

# Keep only production column
data_prod_keep = data_prod_keep.loc[(data_prod_keep.element=='Production'),:].reset_index(drop=True)
data_prod_keep= data_prod_keep.drop(columns =['area_code_(m49)','item_code_(cpc)','element_code','year_code','flag','note'])

# Check temporal resolution of data: 
group_y= data_prod_keep.groupby(by='item_code').apply(lambda group : len(group.year.unique()))
print(group_y.value_counts())

# Drop low temporal resolution products (for tonnes): "Rubber..." and "Butter goat milk"
data_prod_keep= data_prod_keep[~data_prod_keep.item_code.isin([839,1022])] 
data_trade_keep= data_trade_keep[data_trade_keep.item_code.isin(data_prod_keep.item_code.unique())]  
# 216 products that have complete temporal ressolution beteween 1986-2021

36    216
5       1
7       1
Name: count, dtype: int64


In [15]:
data_trade_keep

,index,destin_country_ISO,origin_country_ISO,item,item_code,year,unit,value,origin_country,destin_country,L1_foodex,Food_group
0,0,BY,UY,"Edible offals of horses and other equines, fr...",1098,2013,tonnes,7.0,Uruguay,Belarus,A01QR,Meat and meat products
1,1,BY,UY,"Edible offals of horses and other equines, fr...",1098,2013,1000 US$,8.0,Uruguay,Belarus,A01QR,Meat and meat products
2,2,BY,UY,"Horse meat, fresh or chilled",1097,2019,tonnes,49.6,Uruguay,Belarus,A01QR,Meat and meat products
3,3,BY,UY,"Horse meat, fresh or chilled",1097,2019,1000 US$,124.0,Uruguay,Belarus,A01QR,Meat and meat products
8,8,BY,UY,Wine,564,2005,tonnes,7.0,Uruguay,Belarus,A03LZ,Alcoholic beverages
...,...,...,...,...,...,...,...,...,...,...,...,...
28200280,32425973,ER,EE,Skim milk and whey powder,898,2000,1000 US$,127.0,Estonia,Eritrea,A02LR,Milk and dairy products
28200287,32425980,ER,BR,"Meat of chickens, fresh or chilled",1058,2004,tonnes,24.0,Brazil,Eritrea,A01QR,Meat and meat products
28200288,32425981,ER,BR,"Meat of chickens, fresh or chilled",1058,2006,tonnes,26.0,Brazil,Eritrea,A01QR,Meat and meat products
28200289,32425982,ER,BR,"Meat of chickens, fresh or chilled",1058,2004,1000 US$,18.0,Brazil,Eritrea,A01QR,Meat and meat products


In [16]:
len(data_trade_keep.item.unique())

pickle.dump(data_trade_keep, open('../Data/Data_trade_filt_production.pkl', 'wb'))


## Explore most traded products

In [45]:
# Do top for 2019
data_trade = pd.read_pickle('../Data/Data_trade_filt_production.pkl').reset_index()  #drop =True

data_trade_2019= data_trade.loc[data_trade.year==2019,:]
data_item_sum = data_trade.groupby(['item','unit'])['value'].sum()

In [46]:
# Pareto rule: 80% of the trade on the hands of 20%. 90%trade = 46 products, 80%trade = 21 products

sort_products = data_item_sum.xs('tonnes', level='unit').sort_values(ascending =False)
sort_products

#
cum_fraction = 0.8
relative_cumsum = (sort_products/sort_products.sum()).cumsum()

sort_products= pd.DataFrame(sort_products)
sort_products['in_thr'] = 'blue'
sort_products.loc[relative_cumsum<=cum_fraction,'in_thr']='orange'
# Keep subset
subset_products_all = sort_products[relative_cumsum<=cum_fraction]
#subset_products_tonnes = sort_products[relative_cumsum<=cum_fraction]

#pickle.dump(subset_products, open('../Data/Data_trade_filt_production_top.pkl', 'wb'))


# 80% tonnes : 21 products. 
# 80% dollars: 45 products
# 20/21 products in tonnes are also in the 80% for dollars, the only one that is not there is "Molasses".

In [51]:
set(subset_products.index).difference(subset_products_all.index)

{'Rapeseed or canola oil, crude', 'Tomatoes'}

In [4]:
print('Subset of products with ',cum_fraction*100,'% of trade (tonnes): ',len(subset_products))
print('Fraction of all products: ',100*len(subset_products)/len(sort_products),'%')

NameError: name 'cum_fraction' is not defined

In [ ]:
sort_products['cumsum_val'] = sort_products.value.cumsum()
sort_products


In [ ]:
# 
fig, ax = plt.subplots (figsize=(10,5))
ax.bar(sort_products.index,sort_products.cumsum_val,color =sort_products.in_thr)
ax.tick_params(axis='x', labelrotation=90)
ax.tick_params(right= True,top= True,left= True, bottom= True)

ax.set_title('Food items total exports (Tonnes)')
#ax.axhline(1e6,color='dimgray')
ax.set_xticklabels([])

In [ ]:
fig, ax = plt.subplots (figsize=(10,5))
ax.bar(sort_products.index,sort_products.value,color =sort_products.in_thr)
ax.tick_params(axis='x', labelrotation=90)
ax.tick_params(right= True,top= True,left= True, bottom= True)

ax.set_title('Food items total exports (Tonnes)')
#ax.axhline(1e6,color='dimgray')
ax.set_xticklabels([])

plt.show()

In [ ]:
fig, ax = plt.subplots (1)
ax.hist(sort_products,bins =500)


In [ ]:
trheshold = 1e6
sort_products = pd.DataFrame(data_item_sum.xs('tonnes', level='unit').sort_values(ascending =False))
sort_products['above_thr'] = 'blue'
sort_products.loc[sort_products.value>= trheshold,'above_thr'] = 'orange'

fig, ax = plt.subplots (figsize=(10,5))
ax.bar(sort_products.index,sort_products.value,color =sort_products.above_thr)
ax.tick_params(axis='x', labelrotation=90)
ax.tick_params(right= True,top= True,left= True, bottom= True)

ax.set_title('Food items total exports (Tonnes)')
#ax.axhline(1e6,color='dimgray')
ax.set_xticklabels([])

plt.show()

In [ ]:
#Check top 50 products:



In [ ]:
len(sort_products[sort_products<1e6])

In [ ]:
more1m = sort_products[sort_products>=1e6]

In [ ]:
len(more1m)